# BUILD THE NEURAL NETWORK

[Tutorials > Optimizing Model Parameters](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#optimization-loop)

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [14]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [15]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [16]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [18]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Hyperparameters

In [19]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [20]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

# 1min31.2s for 10 epochs on CPU

Epoch 1
-------------------------------
loss: 2.289927  [   64/60000]
loss: 2.283978  [ 6464/60000]
loss: 2.263356  [12864/60000]
loss: 2.267307  [19264/60000]
loss: 2.247371  [25664/60000]
loss: 2.204269  [32064/60000]
loss: 2.221738  [38464/60000]
loss: 2.176696  [44864/60000]
loss: 2.175632  [51264/60000]
loss: 2.148150  [57664/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 2.144040 

Epoch 2
-------------------------------
loss: 2.149864  [   64/60000]
loss: 2.139866  [ 6464/60000]
loss: 2.080721  [12864/60000]
loss: 2.107498  [19264/60000]
loss: 2.044075  [25664/60000]
loss: 1.971353  [32064/60000]
loss: 2.007968  [38464/60000]
loss: 1.914380  [44864/60000]
loss: 1.927942  [51264/60000]
loss: 1.854799  [57664/60000]
Test Error: 
 Accuracy: 50.8%, Avg loss: 1.856564 

Epoch 3
-------------------------------
loss: 1.886970  [   64/60000]
loss: 1.854261  [ 6464/60000]
loss: 1.736931  [12864/60000]
loss: 1.794299  [19264/60000]
loss: 1.668931  [25664/60000]
loss: 1.615279  [32064/600